In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
wind_turbines = pd.read_csv('../original_datasets/wind_turbine.csv', engine='python',encoding='latin1')

In [3]:
wind_turbines.isnull().sum()

case_id              0
faa_ors           5792
faa_asn           5122
usgs_pr_id       32545
eia_id            5793
t_state              0
t_county             0
t_fips               0
p_name               0
p_year             613
p_tnum               0
p_cap             4482
t_manu            5640
t_model           5777
t_cap             5480
t_hh              6180
t_rd              5934
t_rsa             5934
t_ttlh            6180
retrofit             0
retrofit_year    64822
t_conf_atr           0
t_conf_loc           0
t_img_date        8316
t_img_srce           0
xlong                0
ylat                 0
dtype: int64

In [4]:
df_list = []
table_list = []
for i in range(10):
    var_name = f'schedule_{i}'
    globals()[var_name] = pd.read_excel('../original_datasets/EIA923_Schedules_2_3_4_5_M_12_2020_Final_Revision.xlsx', i)
    table_list.append(var_name)
    df_list.append(globals()[var_name])
table_list

['schedule_0',
 'schedule_1',
 'schedule_2',
 'schedule_3',
 'schedule_4',
 'schedule_5',
 'schedule_6',
 'schedule_7',
 'schedule_8',
 'schedule_9']

In [5]:
def count_nulls():
    return [f'{f"schedule_{i}"} has a total of {sum(globals()[f"schedule_{i}"].isnull().sum())} nulls' for i in range(7)]

In [6]:
count_nulls()

['schedule_0 has a total of 23496 nulls',
 'schedule_1 has a total of 750 nulls',
 'schedule_2 has a total of 813 nulls',
 'schedule_3 has a total of 143 nulls',
 'schedule_4 has a total of 573 nulls',
 'schedule_5 has a total of 342 nulls',
 'schedule_6 has a total of 130 nulls']

In [7]:
def table_cleaning():
    for i in range(10):
        var_name = f"schedule_{i}"
        globals()[var_name] = globals()[var_name].tail(-3)

In [8]:
schedule_1 = schedule_1.tail(int(str(schedule_1[schedule_1['U.S. Department of Energy, The Energy Information Administration (EIA)'] == 'Plant Id'].index).split(']')[0].split('[')[-1])*-1)
schedule_1.columns = schedule_1.iloc[0]
schedule_1 = schedule_1[1:].reset_index(drop=True)
schedule_1.columns = schedule_1.columns.str.replace('\n', ' ').str.lower().str.replace(' ', '_')

In [9]:
schedule_1.columns = schedule_1.columns.str.replace('\n', ' ').str.lower().str.replace(' ', '_')
schedule_1.isnull().sum()

5
plant_id                              0
combined_heat_and_power_plant         0
nuclear_unit_id                       0
plant_name                            0
operator_name                         0
                                     ..
electric_fuel_consumption_quantity    0
total_fuel_consumption_mmbtu          0
elec_fuel_consumption_mmbtu           0
net_generation_(megawatthours)        0
year                                  0
Length: 97, dtype: int64

In [10]:
schedule_1.columns

Index(['plant_id', 'combined_heat_and_power_plant', 'nuclear_unit_id',
       'plant_name', 'operator_name', 'operator_id', 'plant_state',
       'census_region', 'nerc_region', 'reserved', 'naics_code',
       'eia_sector_number', 'sector_name', 'reported_prime_mover',
       'reported_fuel_type_code', 'aer_fuel_type_code', 'reserved',
       'respondent_frequency', 'physical_unit_label', 'quantity_january',
       'quantity_february', 'quantity_march', 'quantity_april', 'quantity_may',
       'quantity_june', 'quantity_july', 'quantity_august',
       'quantity_september', 'quantity_october', 'quantity_november',
       'quantity_december', 'elec_quantity_january', 'elec_quantity_february',
       'elec_quantity_march', 'elec_quantity_april', 'elec_quantity_may',
       'elec_quantity_june', 'elec_quantity_july', 'elec_quantity_august',
       'elec_quantity_september', 'elec_quantity_october',
       'elec_quantity_november', 'elec_quantity_december',
       'mmbtuper_unit_january',

In [51]:
wind_data = pd.read_excel('../original_datasets/EIA923_Schedules_2_3_4_5_M_12_2020_Final_Revision.xlsx', 0)

wind_data = wind_data.tail(int(str(wind_data[wind_data['U.S. Department of Energy, The Energy Information Administration (EIA)'] == 'Plant Id'].index).split(']')[0].split('[')[-1])*-1)
wind_data.columns = wind_data.iloc[0]
wind_data = wind_data[1:].reset_index(drop=True)
wind_data.columns = wind_data.columns.str.replace('\n', ' ').str.lower().str.replace(' ', '_')

wind_data = wind_data[['plant_id', 'plant_name', 'operator_name', 'operator_id', 'plant_state', 'census_region', 'reported_prime_mover', 'reported_fuel_type_code', 
                       'total_fuel_consumption_mmbtu', 'net_generation_(megawatthours)']]

wind_data.rename(columns={'operator_name':'op', 'operator_id':'op_id', 'plant_name':'plant', 'census_region':'region', 'reported_prime_mover':'rpm', 
                               'reported_fuel_type_code':'fuel_type', 'net_generation_(megawatthours)':'net_gen', 'total_fuel_consumption_mmbtu':'fuel_usage'}, inplace=True)

wind_data.loc[(wind_data['net_gen']!=0)]
wind_data = wind_data[wind_data['fuel_type'] == 'WND']
wind_data.drop(columns=('fuel_type'), inplace=True)

for col in wind_data.iloc[:, 8:]:
    wind_data[col] = pd.to_numeric(wind_data[col])
    wind_data[col] = wind_data[col].fillna(0)

wind_data.reset_index(inplace=True, drop=True)
wind_data.to_csv('../cleaned_datasets/wind_data.csv')
wind_data

4,plant_id,plant,op,op_id,plant_state,region,rpm,fuel_usage,net_gen
0,1,Sand Point,"TDX Sand Point Generating, LLC",63560,AK,PACN,WT,3045,347.000
1,90,Snake River,Nome Joint Utility Systems,13642,AK,PACN,WT,20906,2383.000
2,508,Lamar Plant,City of Lamar - (CO),10633,CO,MTN,WT,116085,13232.000
3,692,Medicine Bow,SRIV Partnership LLC,62042,WY,MTN,WT,141429,16121.000
4,944,Geneseo,City of Geneseo - (IL),7095,IL,ENC,WT,51368,5855.000
...,...,...,...,...,...,...,...,...,...
1238,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,NV,MTN,WT,279086,31811.979
1239,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,MD,SAT,WT,76102,8674.526
1240,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,NH,NEW,WT,84267,9605.242
1241,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,MA,NEW,WT,6973,794.787


In [ ]:
generator_data = pd.read_excel('../original_datasets/EIA923_Schedules_2_3_4_5_M_12_2020_Final_Revision.xlsx', 0)

In [ ]:
generator_data = generator_data.tail(int(str(generator_data[generator_data['U.S. Department of Energy, The Energy Information Administration (EIA)'] == 'Plant Id'].index).split(']')[0].split('[')[-1])*-1)
generator_data.columns = generator_data.iloc[0]
generator_data = generator_data[1:].reset_index(drop=True)
generator_data.columns = generator_data.columns.str.replace('\n', ' ').str.lower().str.replace(' ', '_')

In [ ]:
generator_data = generator_data[['plant_id', 'plant_name', 'operator_name', 'operator_id', 'plant_state', 'census_region', 'reported_prime_mover', 'reported_fuel_type_code', 'quantity_january',
       'quantity_february', 'quantity_march', 'quantity_april', 'quantity_may', 'quantity_june', 'quantity_july', 'quantity_august', 'quantity_september', 'quantity_october', 
                                 'quantity_november', 'quantity_december', 'netgen_january', 'netgen_february', 'netgen_march', 'netgen_april', 'netgen_may', 'netgen_june', 'netgen_july', 'netgen_august', 
                'netgen_september', 'netgen_october', 'netgen_november', 'netgen_december', 'net_generation_(megawatthours)']]
generator_data.rename(columns={'operator_name':'op', 'operator_id':'op_id', 'plant_name':'plant', 'census_region':'region', 'reported_prime_mover':'rpm', 
                               'reported_fuel_type_code':'fuel_type', 'net_generation_(megawatthours)':'net_gen'}, inplace=True)

In [ ]:
generator_data = generator_data[generator_data['fuel_type'] == 'WND']
generator_data.drop(columns=('fuel_type'), inplace=True)
generator_data

In [ ]:
quan = generator_data.iloc[:, [7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]]
for col in quan:
    quan.rename(columns={col:f'quan_{str(quan.columns.get_loc(col) + 1)}'}, inplace=True)
quan

In [ ]:
net_gen = generator_data.iloc[:, [19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]]
for col in net_gen:
    net_gen.rename(columns={col:f'netgen_{str(net_gen.columns.get_loc(col) + 1)}'}, inplace=True)
net_gen

In [ ]:
generator_data = pd.merge(left = generator_data.iloc[:, :7], right = generator_data.loc[:, ['plant_id', 'net_gen', 'year']], how='left', on='plant_id')
monthly_totals = quan.join(net_gen)
generator_data = generator_data.join(monthly_totals)
generator_data.columns

In [ ]:
generator_data

In [ ]:
generator_data['net_gen'] = pd.to_numeric(generator_data['net_gen'])
for col in generator_data.iloc[:, 7:]:
    generator_data[col] = pd.to_numeric(generator_data[col])
    generator_data[col] = generator_data[col].fillna(0)
generator_data

In [14]:
manu_data = wind_turbines[['t_manu','t_model','t_cap','t_hh','t_rd','t_rsa','t_ttlh','t_conf_atr']]

In [15]:
manu_data.dropna(inplace=True)

C:\Users\evana\AppData\Local\Temp\ipykernel_952\2803359016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manu_data.dropna(inplace=True)


In [23]:
wind_turbines = wind_turbines.drop(['faa_ors','faa_asn','usgs_pr_id','eia_id','t_conf_loc','t_img_date','t_img_srce','retrofit_year',
                                   't_manu','t_model','t_cap','t_hh','t_rd','t_rsa','t_ttlh','t_conf_atr'], axis = 1)

In [24]:
wind_turbines.head()

,case_id,t_state,t_county,t_fips,p_name,p_year,p_tnum,p_cap,retrofit,xlong,ylat
0,3072661,CA,Kern County,6029,251 Wind,1987.0,194,18.43,0,-118.363762,35.077908
1,3072695,CA,Kern County,6029,251 Wind,1987.0,194,18.43,0,-118.364410,35.077435
2,3072704,CA,Kern County,6029,251 Wind,1987.0,194,18.43,0,-118.364197,35.077644
3,3063272,IA,Story County,19169,30 MW Iowa DG Portfolio,2017.0,10,30.00,0,-93.430367,42.028233
4,3053390,IA,Boone County,19015,30 MW Iowa DG Portfolio,2017.0,10,30.00,0,-93.700424,41.977608


In [39]:
def fix_ints(row):
    if len(row) < 5:
        outcome = '0' + str([t_fips])
    else:
        outcome = str(row[t_fips])
    return outcome

In [40]:
wind_turbines['t_fips'] = wind_turbines['t_fips'].apply(fix_ints, axis=1)
wind_turbines

NameError: name 't_fips' is not defined

In [19]:
wind_turbines
wind_turbines.to_csv('../cleaned_datasets/wind_turbines.csv')

In [17]:
manu_data.reset_index(drop=True)
manu_data.to_csv('../cleaned_datasets/manu_data.csv')

In [45]:
total_energy = pd.read_excel('../original_datasets/EIA923_Schedules_2_3_4_5_M_12_2020_Final_Revision.xlsx', 0)

total_energy = total_energy.tail(int(str(total_energy[total_energy['U.S. Department of Energy, The Energy Information Administration (EIA)'] == 'Plant Id'].index).split(']')[0].split('[')[-1])*-1)
total_energy.columns = total_energy.iloc[0]
total_energy = total_energy[1:].reset_index(drop=True)
total_energy.columns = total_energy.columns.str.replace('\n', ' ').str.lower().str.replace(' ', '_')

total_energy = total_energy[['plant_id', 'plant_name', 'operator_name', 'operator_id', 'plant_state', 'census_region', 'reported_prime_mover', 'reported_fuel_type_code', 
                             'total_fuel_consumption_mmbtu', 'net_generation_(megawatthours)']]

total_energy.rename(columns={'operator_name':'op', 'operator_id':'op_id', 'plant_name':'plant', 'census_region':'region', 'reported_prime_mover':'rpm', 
                               'reported_fuel_type_code':'fuel_type', 'net_generation_(megawatthours)':'net_gen', 'total_fuel_consumption_mmbtu':'fuel_usage'}, inplace=True)

for col in total_energy.iloc[:, 8:]:
    total_energy[col] = pd.to_numeric(total_energy[col])
    total_energy[col] = total_energy[col].fillna(0)

total_energy.loc[(total_energy['net_gen']!=0)]

total_energy['fuel_usage'] = round(total_energy['fuel_usage'] * 0.29307107017222, 3)
    
total_energy.to_csv('../cleaned_datasets/total_energy.csv')
total_energy

4,plant_id,plant,op,op_id,plant_state,region,rpm,fuel_type,fuel_usage,net_gen
0,1,Sand Point,"TDX Sand Point Generating, LLC",63560,AK,PACN,WT,WND,8.924010e+02,347.000
1,2,Bankhead Dam,Alabama Power Co,195,AL,ESC,HY,WAT,3.578219e+05,139170.000
2,3,Barry,Alabama Power Co,195,AL,ESC,CA,NG,3.645376e+05,2633517.000
3,3,Barry,Alabama Power Co,195,AL,ESC,CT,NG,1.553475e+07,5084350.000
4,3,Barry,Alabama Power Co,195,AL,ESC,ST,BIT,7.675155e+06,2625834.500
...,...,...,...,...,...,...,...,...,...,...
15110,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,NV,MTN,WT,WND,8.179203e+04,31811.979
15111,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,MD,SAT,WT,WND,2.230329e+04,8674.526
15112,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,NH,NEW,WT,WND,2.469622e+04,9605.242
15113,99999,State-Fuel Level Increment,State-Fuel Level Increment,99999,MA,NEW,WT,WND,2.043585e+03,794.787


In [ ]:
print(total_energy['fuel_usage'].mean())
print(total_energy['net_gen'].mean())

In [ ]:
total_energy[total_energy['fuel_usage'] <= total_energy['net_gen']]